<a href="https://colab.research.google.com/github/cbikkanur/twitter/blob/master/documentSimilarity_BERT_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
import csv
import os
import time
import datetime
print(os.listdir("."))
from scipy.spatial import distance

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
np.random.seed(0)

import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import warnings
warnings.filterwarnings('ignore')


['.config', 'sample_data']
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
import tensorflow as tf
# Go to Edit > Notebook Settings > Hardware accelerator > GPU
device_name = tf.test.gpu_device_name()
print(device_name)

import torch
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('We will use {} {}GPU(s)'.format(torch.cuda.device_count(), torch.cuda.get_device_name() ))
else:
  device = torch.device('cuda')
  print('No GPU(s) available; we will use "cpu"')

device=torch.device('cuda')

/device:GPU:0
We will use 1 Tesla T4GPU(s)


In [3]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
DRIVE_PATH = "/content/drive/My Drive/MIDS/docSimilarity/"

print(os.listdir(DRIVE_PATH))

['CosineSimilarity.ipynb', 'getTwitterData_TweePy.ipynb', 'cbikkanur_twitter_credentials.json', 'cbikkanur_twitter_credentials.ipynb', 'tweets_04232020.csv', 'SearchTweets.csv', 'BankTweets.csv', 'documentSimilarity.ipynb', 'tweets_04242020.csv', 'SearchTweets_04242020.csv', 'BankTweets_04242020.csv', 'SearchTweets_04252020.csv', 'tweets_04252020.csv', 'BankTweets_04252020.csv', 'documentSimilarity_TfIdf.ipynb.txt', 'SearchTweets_04262020.csv', 'BankTweets_04262020.csv', 'tweets_04262020.csv', '.ipynb_checkpoints', 'model_save', 'train_vectors.npy', 'test_vectors.npy', 'getTwitterData.ipynb', 'SearchTweets_04292020.csv', 'BankTweets_04292020.csv', 'tweets_04292020.csv', 'BankTweets_04302020.csv', 'tweets_04302020.csv', 'SearchTweets_04302020.csv', 'TfIdf_similar_test_tweets.csv', 'documentSimilarity_TfIdf.ipynb']


In [7]:
all_files = [DRIVE_PATH+'tweets_04232020.csv', DRIVE_PATH+'./tweets_04242020.csv', DRIVE_PATH+'./tweets_04252020.csv',
             DRIVE_PATH+'./tweets_04262020.csv', DRIVE_PATH+'./tweets_04292020.csv', DRIVE_PATH+'./tweets_04302020.csv']

df_list = []
for filename in all_files:
    df_list.append(pd.read_csv(filename, sep=',',header=0, encoding='utf-8', index_col = 0))
    
df_tweets = pd.concat(df_list, ignore_index=True)
df_tweets.head()   

,Tweet_Id,User_Id,User_Name,User_Screen_Name,Theme,Text
0,1253375593271394312,80374332,Citibank,Citibank,Bank/Financial,Registering for online access and activating y...
1,1253322747851296768,80374332,Citibank,Citibank,Bank/Financial,Protect your CARES Act payments: Validate comm...
2,1253066045700681731,80374332,Citibank,Citibank,Bank/Financial,It’s simple to set up a payment account in the...
3,1252960358454767616,80374332,Citibank,Citibank,Bank/Financial,We’re committed to helping provide the support...
4,1252673469852143618,80374332,Citibank,Citibank,Bank/Financial,Protect Yourself from COVID-19 Scams: Don’t re...


In [9]:
df_tweets.drop_duplicates(["Text"], inplace=True) # remove duplicates in place and reset index
df_tweets = df_tweets.reset_index(drop=True)
df_tweets.shape

(7700, 6)

## Preprocessing:

In [10]:
query_index = 10
df = pd.DataFrame()
df['text'] = df_tweets['Text']
print('{:=^100}\n\n {}'.format(' Raw text ', df.text.loc[query_index])) 

df['text'] = df['text'].str.replace('http\S+', '') # removing URLs
df['text'] = df['text'].str.replace('[^A-Za-z0-9]+', ' ') # retain only alphanumeric
df['text'] = df['text'].map(lambda x: WordNetLemmatizer().lemmatize(x)) # lemmatization
df['text'] = df['text'].map(lambda x: x.lower()) # to lower case

print('\n{:=^100}\n\n {}'.format(' Preprocessed text ', df.text.loc[query_index])) 

============================================= Raw text =============================================

 To our heroes going out into the world each day – thank you. Citi is supporting COVID-19 relief efforts around the world to help those bravely showing up for the rest of us. https://t.co/oaknGpLLlX https://t.co/wRZ7o6lupM

======================================== Preprocessed text =========================================

 to our heroes going out into the world each day thank you citi is supporting covid 19 relief efforts around the world to help those bravely showing up for the rest of us 


In [11]:
total_indices_size = df.shape[0]
train_size = 0.8
train_indices_size = int(train_size * total_indices_size)
test_indices_size = total_indices_size - train_indices_size
print('total records: {}\n trainig_set record: {}\n test_set records: {}'.format(total_indices_size, train_indices_size, test_indices_size))

total_indices_array = np.array([x for x in range(total_indices_size)])
np.random.shuffle(total_indices_array)
train_indices, test_indices = total_indices_array[:train_indices_size], total_indices_array[train_indices_size:]

total records: 7700
 trainig_set record: 6160
 test_set records: 1540


In [12]:
train_set, test_set = df.loc[train_indices], df.loc[test_indices]
train_set[:5]

,text
2241,penelopeonthego lindseyboylan heerjeet perfec...
4644,hey fedex have you seen my fancy coffee machine
6446,promote to team 4 akb48 at 23 march 2012 and m...
7406,rt cbcolympics this canadian athlete will be h...
912,kllingboys the feeling is mutual talk about a...


In [13]:
# Documentation for transformers: https://huggingface.co/transformers/index.html
! pip install transformers

     |████████████████████████████████| 573kB 9.4MB/s 
     |████████████████████████████████| 1.0MB 61.2MB/s 
     |████████████████████████████████| 890kB 62.1MB/s 
     |████████████████████████████████| 3.7MB 44.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=e990661ffc974f3136145f0007b718b97e10c444f87ca9f4a5c153c9d8e13f6a
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [14]:
import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from keras.preprocessing.sequence import pad_sequences

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', output_hidden_states = True ) # return hidden layers
model.cuda

Using TensorFlow backend.


<bound method Module.cuda of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNo

In [0]:
def vectorize_sentence(tokenizer, model, text):
  MAX_LENGTH = 100
  input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=MAX_LENGTH) # encode for tokenizing
  padded_sequences = pad_sequences([input_ids], maxlen=MAX_LENGTH, dtype='long', truncating='post', padding='post') # pad 0 at the end of sentence until MAX_LENGTH
  input_ids = padded_sequences[0] # remove the outer list
  attention_mask = [int(i>0) for i in input_ids]

  input_ids = torch.tensor(input_ids)
  attention_mask = torch.tensor(attention_mask)

  input_ids = input_ids.unsqueeze(0) # Extra dimension for batch
  attention_mask = attention_mask.unsqueeze(0)  # Extra dimension for batch

  model.eval() # Only forward pass
  input_ids.to(device) # https://github.com/huggingface/transformers/blob/master/notebooks/02-transformers.ipynb
  attention_mask.to(device)

  with torch.no_grad(): # No gradient descent n
    logits, encoded_layers = model(input_ids = input_ids, token_type_ids = None, attention_mask = attention_mask)
    last_layer = 12
    batch = 0
    last_token_index = 0 

    vector = encoded_layers[last_layer][batch][last_token_index]
    vector = vector.detach().cpu().numpy()

    return vector

In [21]:
sample_text = test_set.iloc[1].text
sample_tokens = tokenizer.tokenize(sample_text)
print('sample text: \n{} \n\n BERT tokens for sample text: \n{}'.format(sample_text, sample_tokens))

sample text: 
 candicontreras7 hello thank you for tweeting us so that we can better assist which u s issued amex card product do you have please do not release any personal identifiable information jad 

 BERT tokens for sample text: 
['candi', '##con', '##tre', '##ras', '##7', 'hello', 'thank', 'you', 'for', 't', '##wee', '##ting', 'us', 'so', 'that', 'we', 'can', 'better', 'assist', 'which', 'u', 's', 'issued', 'am', '##ex', 'card', 'product', 'do', 'you', 'have', 'please', 'do', 'not', 'release', 'any', 'personal', 'identifiable', 'information', 'ja', '##d']


In [22]:
sample_vector = vectorize_sentence(tokenizer, model, sample_text)
print('Shape of the vector: {}'.format(sample_vector.shape))

Shape of the vector: (768,)


In [0]:
def format_time(elapsed):
  return str(datetime.timedelta(seconds = int(round(elapsed))))


In [0]:
def vectorize_dataset(df):
  start_time = time.time()
  embeddings = []
  num_records = df.shape[0]
  row_num = 0
  for index, row in df.iterrows():
    text = row['text']
    if row_num % 200 == 0 and not row_num == 0:
      elapsed = format_time(time.time() - start_time)
      rows_per_sec = (time.time() - start_time) / row_num
      remaining_sec = rows_per_sec * (num_records - row_num)
      remaining = format_time(remaining_sec)
      print(' comment {:>7,} of {:>7,}.    Elapsed: {:}. Remaining: {:}'.format(row_num, num_records, elapsed, remaining))
    embeddings.append(vectorize_sentence(tokenizer, model, text))
    row_num += 1
  return  embeddings 

In [25]:
train_set_embeddings = vectorize_dataset(train_set)
train_vectors = np.stack(train_set_embeddings)
train_vectors.shape

 comment     200 of   6,160.    Elapsed: 0:00:41. Remaining: 0:20:07
 comment     400 of   6,160.    Elapsed: 0:01:21. Remaining: 0:19:26
 comment     600 of   6,160.    Elapsed: 0:02:01. Remaining: 0:18:44
 comment     800 of   6,160.    Elapsed: 0:02:41. Remaining: 0:18:01
 comment   1,000 of   6,160.    Elapsed: 0:03:21. Remaining: 0:17:19
 comment   1,200 of   6,160.    Elapsed: 0:04:01. Remaining: 0:16:37
 comment   1,400 of   6,160.    Elapsed: 0:04:41. Remaining: 0:15:54
 comment   1,600 of   6,160.    Elapsed: 0:05:20. Remaining: 0:15:13
 comment   1,800 of   6,160.    Elapsed: 0:05:59. Remaining: 0:14:31
 comment   2,000 of   6,160.    Elapsed: 0:06:39. Remaining: 0:13:50
 comment   2,200 of   6,160.    Elapsed: 0:07:18. Remaining: 0:13:09
 comment   2,400 of   6,160.    Elapsed: 0:07:58. Remaining: 0:12:28
 comment   2,600 of   6,160.    Elapsed: 0:08:38. Remaining: 0:11:49
 comment   2,800 of   6,160.    Elapsed: 0:09:16. Remaining: 0:11:07
 comment   3,000 of   6,160.    El

(6160, 768)

In [26]:
test_set_embeddings = vectorize_dataset(test_set)
test_vectors = np.stack(test_set_embeddings)
test_vectors.shape


 comment     200 of   1,540.    Elapsed: 0:00:38. Remaining: 0:04:14
 comment     400 of   1,540.    Elapsed: 0:01:16. Remaining: 0:03:37
 comment     600 of   1,540.    Elapsed: 0:01:54. Remaining: 0:02:59
 comment     800 of   1,540.    Elapsed: 0:02:32. Remaining: 0:02:21
 comment   1,000 of   1,540.    Elapsed: 0:03:10. Remaining: 0:01:43
 comment   1,200 of   1,540.    Elapsed: 0:03:48. Remaining: 0:01:05
 comment   1,400 of   1,540.    Elapsed: 0:04:26. Remaining: 0:00:27


(1540, 768)

In [27]:
output_dir = './drive/My Drive/MIDS/docSimilarity/model_save/'
print(os.listdir("."))
if not os.path.exists(output_dir):
  print('directory does not exist')
  os.makedirs(output_dir)

np.save('train_vectors_alnum.npy', train_vectors)  
np.save('test_vectors_alnum.npy', test_vectors)  

! cp -r ./train_vectors_alnum.npy './drive/My Drive/MIDS/docSimilarity/model_save/'
! cp -r ./test_vectors_alnum.npy './drive/My Drive/MIDS/docSimilarity/model_save/'

['.config', 'drive', 'sample_data']


# Cosine Similarity:

In [0]:
def print_similar_texts(query_text, query_text_vector,  train_matrix, train_indices, df_tweets, n = 5):
    cosine_similarities_n = cosine_similarity(query_text_vector, train_matrix).flatten()
    top_Indices = cosine_similarities_n.argsort()[::-1][:n]
    top_tweet_Indices = train_indices[top_Indices]
    
    print('\nInput Text:\n {} \n'.format(query_text))
    for index, sim_text in enumerate(df_tweets.loc[top_tweet_Indices, "Text"]):
        print('=' * 30, 'Similar Text: {} || Similar Score: {}'.format(index+1, np.round(cosine_similarities_n[top_Indices[index]], 3)), '=' * 30, '\n')
        print(sim_text, '\n')

In [29]:
query_index = 77
query_text = df_tweets.loc[test_indices[query_index], "Text"]
query_text_vector = test_vectors[query_index].reshape(1, -1)
print_similar_texts(query_text, query_text_vector, train_vectors, train_indices, df_tweets, n = 5)


Input Text:
 @SasiUgarteTeves We haven't heard back from you. Please let us know if you need further assistance. ^Shaun https://t.co/gvC4jAOA9c 

============================== Similar Text: 1 || Similar Score: 0.9769999980926514 ============================== 

@amj663 We haven't heard back from you. Please let us know if you require further assistance. ^Shaun https://t.co/gvC4jAOA9c 

============================== Similar Text: 2 || Similar Score: 0.9739999771118164 ============================== 

@Anubhav28284 We haven't heard back from you. Please let us know if you require further assistance. ^Shaun https://t.co/gvC4jAOA9c 

============================== Similar Text: 3 || Similar Score: 0.972000002861023 ============================== 

@dbfact We haven't heard back from you. Please send a DM if further help is required. ^Shaun https://t.co/gvC4jAOA9c 

============================== Similar Text: 4 || Similar Score: 0.9710000157356262 ============================== 

@ag3982

In [0]:
def find_similar_n(train_matrix, train_indices, test_matrix, test_indices, test_set_df, df_tweets, n=5):
    similar_texts_list = []
    similar_scores_list = []
    similar_top_score_list = []
    similar_themes_list = []
    similar_top_themes_list = []
    for vector in test_matrix:
        cosine_similarities_n = cosine_similarity(vector.reshape(1, -1), train_matrix).flatten()
        similar_doc_indices =  cosine_similarities_n.argsort()[::-1][:n]
        top_tweet_Indices = train_indices[similar_doc_indices]
        
        similar_texts = [text for text in df_tweets.loc[top_tweet_Indices, "Text"]]
        similar_texts_list.append(similar_texts)
        
        similar_scores = [np.round(score, 3) for score in cosine_similarities_n[similar_doc_indices]]
        similar_scores_list.append(similar_scores)
        similar_top_score_list.append(similar_scores[0])
                                                                   
        similar_themes = [theme for theme in df_tweets.loc[top_tweet_Indices, "Theme"]]
        similar_themes_list.append(similar_themes)
        
        top_similar_theme = max(set(similar_themes), key=lambda x: similar_themes.count(x))
        similar_top_themes_list.append(top_similar_theme)
    
    df = test_set_df.copy() 
    df["original_theme"] = df_tweets.loc[test_indices, "Theme"]
    df["similar_texts"] = similar_texts_list
    df["similar_scores"] = similar_scores_list  
    df["top_similar_score"] = similar_top_score_list
    df["similar_themes"] = similar_themes_list
    df["top_similar_themes"] = similar_top_themes_list
    return df

In [50]:
similar_test_df = find_similar_n(train_vectors, train_indices, test_vectors, test_indices, test_set, df_tweets, n=10)
similar_test_df.head()

,text,original_theme,similar_texts,similar_scores,top_similar_score,similar_themes,top_similar_themes
4648,catnip coffee you dont have to give me a new ...,coffee,[RT @claylongs: Jerking off in the car up on m...,"[0.935, 0.932, 0.929, 0.929, 0.928, 0.927, 0.9...",0.935,"[car, cake, cake, cake, cake, cake, Bank/Finan...",cake
362,candicontreras7 hello thank you for tweeting ...,Bank/Financial,"[@JTrey3 Hello, thank you for tweeting us. So ...","[0.996, 0.995, 0.995, 0.994, 0.992, 0.992, 0.9...",0.996,"[Bank/Financial, Bank/Financial, Bank/Financia...",Bank/Financial
4645,teekhi mirchh harriisqureshi lemme healuh maj...,coffee,[RT @guu_tara: doodle ZERO(ver.metless) https...,"[0.93, 0.922, 0.922, 0.919, 0.918, 0.918, 0.91...",0.930,"[car, tokyo, ramadan, dog, coffee, nasa, cake,...",car
2086,rt trumpwarroom are you kidding me michigan de...,covid-19,[RT @JCAPoquoson: Punished for telling the tru...,"[0.919, 0.915, 0.911, 0.909, 0.908, 0.906, 0.9...",0.919,"[covid-19, covid-19, nasa, dog, dog, covid-19,...",covid-19
2988,state rep whitsett nearly died from coronaviru...,covid-19,[RT @GOPChairwoman: State Rep. Whitsett nearly...,"[0.903, 0.891, 0.89, 0.89, 0.889, 0.888, 0.888...",0.903,"[covid-19, dog, covid-19, @Citi, covid-19, cov...",covid-19


In [51]:
accuracy = 100 * np.sum(similar_test_df["original_theme"] == similar_test_df["top_similar_themes"])/similar_test_df.shape[0]
print("Accuracy on test data in predicting theme: {}%".format(np.round(accuracy,2)))

Accuracy on test data in predicting theme: 72.4%


In [0]:
similar_test_df.to_csv("BERT_similar_test_tweets.csv")

In [0]:
os.listdir('.')
! cp -r ./BERT_similar_test_tweets.csv './drive/My Drive/MIDS/docSimilarity/model_save/'

# Nearest Euclidean:

In [0]:
def print_nearest_euclidean_texts(query_text, query_text_vector,  train_matrix, train_indices, df_tweets, n = 5):
    euclidean_distance = distance.cdist(query_text_vector.reshape(1, -1), train_vectors, 'euclidean').flatten() 
    top_Indices = euclidean_distance.argsort()[:n]
    top_tweet_Indices = train_indices[top_Indices]
    
    print('\nInput Text:\n {} \n'.format(query_text))
    for index, sim_text in enumerate(df_tweets.loc[top_tweet_Indices, "Text"]):
        print('=' * 30, 'Similar Text: {} || Euclidean Distance: {}'.format(index+1, np.round(euclidean_distance[top_Indices[index]], 3)), '=' * 30, '\n')
        print(sim_text, '\n')

In [55]:
query_index = 16
query_text = df_tweets.loc[test_indices[query_index], "Text"]
query_text_vector = test_vectors[query_index]
print_nearest_euclidean_texts(query_text, query_text_vector, train_vectors, train_indices, df_tweets, n = 5)


Input Text:
 @DLoesch Correct. Trump said inject a disinfectant directly into a person, not lungs 🤣. Videos don't lie so pretty sad Dana is full #Trump lapdog today trying to deflect  #CorruptGOP #Cult45 #Covid_19 #TrumpIsAnIdiot 

https://t.co/nwx7n0VHv8 

============================== Similar Text: 1 || Euclidean Distance: 4.391 ============================== 

RT @13chwar: this lucky dog

#WeLoveJennie
#HeartsForJennie 
@ygofficialblink https://t.co/hXmIr2czi1 

============================== Similar Text: 2 || Euclidean Distance: 4.512 ============================== 

@Facetiouslyfun @IttyTater @PumpkinPatch200 @Ilovecanada13 @facepalmchris @MeowMixFan1 @TDenham1106 @chriscoastguy @a_sirup @Kimberly5179 @imnotgonnashare @Cmags1963 @justloo48273802 @StillMeowing @GrumpyLittle1 @Cdnwatcher @criticalandcon @Prissi_coffee @dia2031 @MrsLauraBski @Indieanabones @JediBlonde @sikhofyou1 @JJCrosstrainers @Radtastiical @onionisfinished @lacey19911 @Aquaryan1972 @mirandamohajrn @bericm @Jod

In [0]:
def find_nearest_n(train_matrix, train_indices, test_matrix, test_indices, test_set_df, df_tweets, n=5):
    similar_texts_list = []
    euclidean_distances_list = []
    euclidean_nearest_distance_list = []   
    similar_themes_list = []
    similar_top_themes_list = []
    for vector in test_matrix:
        euclidean_distance = distance.cdist(vector.reshape(1, -1), train_vectors, 'euclidean').flatten()
        nearest_doc_indices =  euclidean_distance.argsort()[:n]        
        top_tweet_Indices = train_indices[nearest_doc_indices]
        
        similar_texts = [text for text in df_tweets.loc[top_tweet_Indices, "Text"]]
        similar_texts_list.append(similar_texts)
        
        euclidean_distances = [np.round(score, 3) for score in euclidean_distance[nearest_doc_indices]]
        euclidean_distances_list.append(euclidean_distances)
        euclidean_nearest_distance_list.append(euclidean_distances[0])
                                                                   
        similar_themes = [theme for theme in df_tweets.loc[top_tweet_Indices, "Theme"]]
        similar_themes_list.append(similar_themes)
        
        top_similar_theme = max(set(similar_themes), key=lambda x: similar_themes.count(x))
        similar_top_themes_list.append(top_similar_theme)
    
    df = test_set_df.copy() 
    df["original_theme"] = df_tweets.loc[test_indices, "Theme"]
    df["similar_texts"] = similar_texts_list
    df["euclidean_distances"] = euclidean_distances_list  
    df["nearest_euclidean_distance"] = euclidean_nearest_distance_list
    df["similar_themes"] = similar_themes_list
    df["top_similar_themes"] = similar_top_themes_list
    return df

In [57]:
nearest_test_df = find_nearest_n(train_vectors, train_indices, test_vectors, test_indices, test_set, df_tweets, n=10)
nearest_test_df.head()

,text,original_theme,similar_texts,euclidean_distances,nearest_euclidean_distance,similar_themes,top_similar_themes
4648,catnip coffee you dont have to give me a new ...,coffee,[RT @claylongs: Jerking off in the car up on m...,"[5.294, 5.377, 5.568, 5.57, 5.62, 5.63, 5.643,...",5.294,"[car, cake, cake, cake, cake, car, cake, car, ...",cake
362,candicontreras7 hello thank you for tweeting ...,Bank/Financial,"[@JTrey3 Hello, thank you for tweeting us. So ...","[1.345, 1.382, 1.505, 1.547, 1.812, 1.855, 2.3...",1.345,"[Bank/Financial, Bank/Financial, Bank/Financia...",Bank/Financial
4645,teekhi mirchh harriisqureshi lemme healuh maj...,coffee,[RT @guu_tara: doodle ZERO(ver.metless) https...,"[5.308, 5.533, 5.619, 5.634, 5.636, 5.735, 5.7...",5.308,"[car, tokyo, ramadan, nasa, dog, coffee, cake,...",ramadan
2086,rt trumpwarroom are you kidding me michigan de...,covid-19,[RT @JCAPoquoson: Punished for telling the tru...,"[5.715, 5.862, 5.976, 6.136, 6.151, 6.167, 6.1...",5.715,"[covid-19, covid-19, nasa, dog, covid-19, rama...",covid-19
2988,state rep whitsett nearly died from coronaviru...,covid-19,[RT @GOPChairwoman: State Rep. Whitsett nearly...,"[6.137, 6.513, 6.519, 6.604, 6.619, 6.647, 6.6...",6.137,"[covid-19, covid-19, dog, dog, @Citi, airlines...",dog


In [58]:
accuracy = 100 * np.sum(nearest_test_df["original_theme"] == nearest_test_df["top_similar_themes"])/nearest_test_df.shape[0]
print("Accuracy on test data in predicting theme: {}%".format(np.round(accuracy,2)))

Accuracy on test data in predicting theme: 72.73%


In [0]:
nearest_test_df.to_csv("BERT_nearest_test_tweets.csv")
! cp -r ./BERT_nearest_test_tweets.csv './drive/My Drive/MIDS/docSimilarity/model_save/'